In [1]:
import pandas as pd

# Carregar o arquivo CSV gerado pelo script
df = pd.read_csv("processed_data_categorized.csv")

# Exibir as primeiras 5 linhas do DataFrame
df.head()

,country_code,type,created_at,content_en,cleaned_content,original_sentiment,cleaned_sentiment,sentiment_difference,validated_sentiment,original_length,cleaned_length,entities,doctor_name,hospital_name,location,category_keywords,category_bert,issue_type,urgency_level,resolution_suggestion
0,br,opinion,2024-04-06,"A doctor who is not very inclusive, it seems t...",doctor not_very inclusive seems want immediate...,-0.9209,-0.9126,0.0083,-0.9126,1018,586,"{'doctor_name': [], 'hospital_name': [], 'loca...",NaN,NaN,NaN,Scheduling Issue,Wait Time Issue,Critical,High,Immediate Action Required
1,br,opinion,2024-04-12,Although the online schedule showed availabili...,although online schedule showed availability t...,0.0000,0.0000,0.0000,0.0000,154,110,"{'doctor_name': [], 'hospital_name': [], 'loca...",NaN,NaN,NaN,Scheduling Issue,Facility Issue,Non-Critical,Low,Monitor
2,br,opinion,2024-04-22,Didn't answer my wife. After traveling more th...,answer wife traveling three hour even though c...,-0.8079,-0.5719,0.2360,-0.5719,246,156,"{'doctor_name': [], 'hospital_name': [], 'loca...",NaN,NaN,NaN,General Issue,Wait Time Issue,Critical,High,Immediate Action Required
3,br,opinion,2024-04-27,\nHe works at the Santo Amaro rehabilitation c...,work santo amaro rehabilitation center not_pre...,-1.0000,-0.8027,0.1973,-0.8027,624,416,"{'doctor_name': [], 'hospital_name': [], 'loca...",NaN,NaN,NaN,Wait Time Issue,Facility Issue,Critical,High,Immediate Action Required
4,br,opinion,2024-04-29,This psychologist canceled the 1st appointment...,psychologist canceled st appointment wa late s...,0.3480,-0.8729,1.2209,-0.8729,636,385,"{'doctor_name': [], 'hospital_name': [], 'loca...",NaN,NaN,NaN,Scheduling Issue,Wait Time Issue,Critical,High,Immediate Action Required


In [2]:
# Verificar os valores únicos na coluna 'issue_type' e contar a quantidade de cada um
issue_type_unique = df['issue_type'].value_counts().reset_index()
issue_type_unique.columns = ['issue_type', 'counts']

# Verificar se a coluna 'dominant_emotion' existe antes de contar os valores únicos
if 'dominant_emotion' in df.columns:
    dominant_emotion_unique = df['dominant_emotion'].value_counts().reset_index()
    dominant_emotion_unique.columns = ['dominant_emotion', 'counts']
else:
    dominant_emotion_unique = None
    print("A coluna 'dominant_emotion' não está presente no DataFrame.")

# Verificar se a coluna 'category' existe antes de contar os valores únicos
if 'category' in df.columns:
    category_unique = df['category'].value_counts().reset_index()
    category_unique.columns = ['category', 'counts']
else:
    category_unique = None
    print("A coluna 'category' não está presente no DataFrame.")

# Exibir os resultados
issue_type_unique, dominant_emotion_unique, category_unique


A coluna 'dominant_emotion' não está presente no DataFrame.
A coluna 'category' não está presente no DataFrame.


(     issue_type  counts
 0  Non-Critical   37840
 1      Critical    7011,
 None,
 None)

In [3]:
# Identificar colunas com dados únicos
unique_columns = df.columns[df.apply(pd.Series.nunique) == 1]

# Identificar colunas com mais de um valor único
non_unique_columns = df.columns[df.apply(pd.Series.nunique) > 1]

# Apresentar a quantidade e os nomes das colunas com dados únicos
print(f"Colunas com dados únicos ({len(unique_columns)}):")
print(unique_columns)

# Apresentar a quantidade e os nomes das colunas com mais de um valor único
print(f"\nColunas com mais de um valor único ({len(non_unique_columns)}):")
print(non_unique_columns)


Colunas com dados únicos (0):
Index([], dtype='object')

Colunas com mais de um valor único (20):
Index(['country_code', 'type', 'created_at', 'content_en', 'cleaned_content',
       'original_sentiment', 'cleaned_sentiment', 'sentiment_difference',
       'validated_sentiment', 'original_length', 'cleaned_length', 'entities',
       'doctor_name', 'hospital_name', 'location', 'category_keywords',
       'category_bert', 'issue_type', 'urgency_level',
       'resolution_suggestion'],
      dtype='object')


In [4]:
# Lista das colunas disponíveis que você mencionou como importantes
columns_to_check = [
    'country_code', 'type', 'created_at', 'content_en', 'cleaned_content',
    'original_sentiment', 'cleaned_sentiment', 'category'
]

# Verificar quais dessas colunas existem no DataFrame
existing_columns = [col for col in columns_to_check if col in df.columns]

# Verificar quais dessas colunas possuem valores únicos
unique_values_check = {col: df[col].nunique() == 1 for col in existing_columns}

# Exibir o resultado
for col, is_unique in unique_values_check.items():
    if is_unique:
        print(f"A coluna '{col}' possui valores únicos.")
    else:
        print(f"A coluna '{col}' possui múltiplos valores.")


A coluna 'country_code' possui múltiplos valores.
A coluna 'type' possui múltiplos valores.
A coluna 'created_at' possui múltiplos valores.
A coluna 'content_en' possui múltiplos valores.
A coluna 'cleaned_content' possui múltiplos valores.
A coluna 'original_sentiment' possui múltiplos valores.
A coluna 'cleaned_sentiment' possui múltiplos valores.


In [5]:
import pandas as pd

# Supondo que o DataFrame seja df
def classify_sentiment(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

# Cria a nova coluna 'sentiment_classification'
df['sentiment_classification'] = df['cleaned_sentiment'].apply(classify_sentiment)

# Agrupa pelo 'sentiment_classification' e conta os registros
sentiment_grouped = df.groupby('sentiment_classification').size().reset_index(name='count')

# Calcula o percentual
total_count = sentiment_grouped['count'].sum()
sentiment_grouped['percentage'] = (sentiment_grouped['count'] / total_count) * 100

# Calcula o acumulado do percentual
sentiment_grouped['cumulative_percentage'] = sentiment_grouped['percentage'].cumsum()

# Exibe o resultado
sentiment_grouped


,sentiment_classification,count,percentage,cumulative_percentage
0,Negative,14928,33.283539,33.283539
1,Neutral,3105,6.922923,40.206461
2,Positive,26818,59.793539,100.000000


In [6]:
import pandas as pd

# Função para agrupar, contar, calcular percentual e acumulado para qualquer coluna
def analyze_column(df, column_name):
    grouped = df[column_name].value_counts().reset_index()
    grouped.columns = [column_name, 'count']

    # Calcula o percentual
    total_count = grouped['count'].sum()
    grouped['percentage'] = (grouped['count'] / total_count) * 100

    # Calcula o acumulado do percentual
    grouped['cumulative_percentage'] = grouped['percentage'].cumsum()

    return grouped

# Lista de todas as colunas no DataFrame
columns_to_analyze = df.columns

# Dicionário para armazenar os resultados de todas as colunas
analysis_results = {}

# Loop para analisar cada coluna e armazenar os resultados
for column in columns_to_analyze:
    analysis_results[column] = analyze_column(df, column)

# Exibir os resultados de todas as colunas
analysis_results


{'country_code':   country_code  count  percentage  cumulative_percentage
 0           br  22111   49.298789              49.298789
 1           es  12950   28.873381              78.172170
 2           de   9790   21.827830             100.000000,
 'type':              type  count  percentage  cumulative_percentage
 0            case  29384   65.514704              65.514704
 1         opinion  15200   33.889991              99.404696
 2            csat    129    0.287619              99.692315
 3  promoter ninja    111    0.247486              99.939801
 4       appfollow     27    0.060199             100.000000,
 'created_at':     created_at  count  percentage  cumulative_percentage
 0   2024-06-12    822    1.832735               1.832735
 1   2024-05-21    786    1.752469               3.585204
 2   2024-06-04    764    1.703418               5.288622
 3   2024-06-05    759    1.692270               6.980892
 4   2024-06-03    757    1.687811               8.668703
 ..         ..